<a href="https://colab.research.google.com/github/DmitryKutsev/NIS_SentiFrame/blob/master/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymorphy2[fast]

     |████████████████████████████████| 51kB 8.9MB/s 
     |████████████████████████████████| 7.1MB 21.3MB/s 
     |████████████████████████████████| 378kB 46.7MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp36-cp36m-linux_x86_64.whl size=857286 sha256=473bb0339e836e5a55f206930776abe0d0249dd0b745eaf2a2b7392c3557ba3c
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG


In [0]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

Check out [this issue](https://github.com/hanxiao/bert-as-service/issues/380) and "make sure Colab is using Tensorflow 1.x, because bert-serving-start doesn't currently work with TF 2.1 and nohup hides the output of the command failing"

In [0]:
# import tensorflow as tf
# print (tf.__version__)

In [0]:
!pip install -U bert-serving-server[http]
!pip install bert-serving-client  # client, independent of `bert-serving-server`

In [0]:
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
!unzip /content/multi_cased_L-12_H-768_A-12.zip

In [0]:
!nohup bert-serving-start -model_dir=./multi_cased_L-12_H-768_A-12 > out.file 2>&1 &

In [0]:
from bert_serving.client import BertClient
bc = BertClient()
encoded_test = bc.encode(['First do it', 'then do it right', 'then do it better'])
encoded_test

In [0]:
encoded_test = bc.encode(['Вася любит Машу'])
encoded_test

In [3]:
# arg0, arg1, verb = 'маша петя любить'.split
def a0_a1_clause_maker(verb, arg0, arg1, arg0_case='nomn', arg1_case='accs', cap=True):

  '''Генерирует простые предложения вида 'arg0 любит arg1'.
  Параметры:
  verb              глагол в любой форме;
  arg0_case='nomn'  тег падежа подлежащего;
  arg1_case='accs'  тег падежа дополнения;
  arg0='Маша'       подлежащее;
  arg1='Петя'       дополнение
  cap=True        капитализация аргументов-имен собственных.
  Допустимые теги падежей лежат тут: http://opencorpora.org/dict.php?act=gram.'''

  inflected_arg0 = morph.parse(arg0)[0].inflect({'sing', 'nomn'}).word
  inflected_verb = morph.parse(verb)[0].inflect({'sing', '3per', 'pres', 'indc'}).word
  inflected_arg1 = morph.parse(arg1)[0].inflect({'sing', 'accs'}).word

  if (cap == True) and ('Name' in morph.parse(arg1)[0].tag):
    inflected_arg1 = inflected_arg1.capitalize()
  if (cap == True) and ('Name' in morph.parse(arg0)[0].tag):
    inflected_arg0 = inflected_arg0.capitalize()

  sentence = '{} {} {}'.format(inflected_arg0, inflected_verb, inflected_arg1)
  return sentence

# a0_a1_clause_maker('любить', 'Маша', 'Петя')
a0_a1_clause_maker('любить', 'Петя', 'Маша')


  

'Петя любит Машу'

In [8]:
# Пете нравится, что
def clause_wrapper(verb, arg0, arg1, 
                   main_subject=1, main_verb='нравиться',
                   arg0_case='nomn', arg1_case='accs', 
                   cap=True):

  '''Генерирует сложноподчинённые предложения вида 'arg нравится, что arg0 любит arg1'.
  Параметры:
  verb              глагол в любой форме;
  arg0_case='nomn'  тег падежа подлежащего подчинённой клаузы;
  arg1_case='accs'  тег падежа дополнения подчинённой клаузы;
  arg0='Маша'       подлежащее подчинённой клаузы;
  arg1='Петя'       дополнение подчинённой клаузы;
  cap=True        капитализация аргументов-имен собственных.
  Допустимые теги падежей лежат тут: http://opencorpora.org/dict.php?act=gram.'''

  if main_subject and ('Name' in morph.parse(arg1)[0].tag):
    inflected_main_subject = morph.parse(arg1)[0].inflect({'sing', 'datv'}).word.capitalize()
  else:
    inflected_main_subject = morph.parse(arg0)[0].inflect({'sing', 'datv'}).word.capitalize()
  
  inflected_main_verb = morph.parse(main_verb)[0].inflect({'sing', '3per', 'pres', 'indc'}).word
  subordinate_clause = a0_a1_clause_maker(verb, arg0, arg1, arg0_case, arg1_case, cap)
  main_clause = '{} {}'.format (inflected_main_subject, inflected_main_verb)
  sentence = '{}, что {}'.format(main_clause, subordinate_clause)

  return sentence

clause_wrapper('любить', 'Петя', 'Маша', 0)


'Пете нравится, что Петя любит Машу'

In [8]:
morph.parse('Пете')[0]

Parse(word='пете', tag=OpencorporaTag('NOUN,anim,masc,Name sing,datv'), normal_form='петя', score=0.5, methods_stack=((<DictionaryAnalyzer>, 'пете', 407, 2),))

In [0]:
# bc.server_status